In [30]:
def getchar(m):
    """returns the next character from the input string m"""
    global p
    p+=1
    return m[p] if p < len(m) else ""

def isdigit(c):
    """returns True if input character c is a digit"""
    Digits = [str(x) for x in range(10)]
    return True if c in Digits else False

def isalpha(c):
    """returns True if input character c is a letter"""
    Uppers = [chr(x) for x in range(ord('A'),ord('Z')+1)]
    Lowers = [chr(x) for x in range(ord('a'),ord('z')+1)]
    return True if c in Uppers or c in Lowers or c == '_' else False

def isalnum(c):
    """returns True if the input character c is a digit or alphabetic"""
    return isdigit(c) or isalpha(c)

def isopFirst(c):
    """return true if input character is the first character of a op or relop"""
    first = ['+', '-', '*', '/', '=', '%', '&', '|', '^', '>', '<', '~', '!', '?']
    return True if c in first else False

"""def isop(c):
    returns True if the input character c is a defined operator
    ops = ['+','-','*','/','=', '%', '&', '|', '^', '~', '!', '?']
    return True if c in ops else False"""

"""def isrelop(c):
    returns True if the input character c is a defined relational operator
    ops = ['<','>']
    return True if c in ops else False"""

def ispun(c):
    """returns True if the input character c is a defined punctuation"""
    puns = ['(',')',';','{','}','[',']', ':', ',']
    return True if c in puns else False

In [31]:
def init():
    """initialize the symbol table for lexical analysis"""
    global p                 # current input position
    p = -1

    global line_num          # current line number
    line_num = 1
    
    global ST                # symbol table
    ST = {}                  # Symbol Table for variables
    ST.setdefault('', 0)
    
    ST["int"] = ("int", "INT")
    ST["float"] = ("float", "FLOAT")
    ST["if"] = ("if","IF")             # adding in reserved words
    ST["else"] = ("else","ELSE")
    ST["for"] = ("for","FOR")
    ST["while"] = ("while","WHILE")
    ST["do"] = ("do","DO")
    ST["break"] = ("break", "BREAK")
    ST["continue"] = ("continue", "CONTINUE")
    ST["goto"] = ("goto", "GOTO")
    
    

    global tokenval          # Value for the current token
    global DONE              # Flag value for end of iteration
    DONE = object()

In [32]:
def lexan(m):
    """modified lexical analyzer from section 2.9, page 74"""
    global p           # string position
    global line_num    # line number
    global ST          # symbol table
    global tokenval    # value of current token
    global DONE        # flag for end of file
    
    tokenval = None    # default value for tokens
    lexeme = ""        # current lexeme being read
    
    while True:
        c = getchar(m)                               # read first character
        
        if c in [' ','\t']:                          # skip over space and tab
            pass
        elif c == '\n':
            line_num += 1                            # increment line counter if newline is read
        
        elif isdigit(c):
            tokenval = int(c)                        # start with integer value if char is a digit
            c = getchar(m)                           # load next character
            while isdigit(c):
                tokenval = 10*tokenval + int(c)      # process next integer if char is a digit
                c = getchar(m)                       # load next character
            p -= 1
            return "NUM"
        
        elif isalpha(c):
            lexeme += c                              # append character to buffer
            c = getchar(m)                           # load next character
            while isalnum(c):
                lexeme += c                          # append character to buffer
                c = getchar(m)                       # load next character
            if lexeme not in ST.keys():
                ST[lexeme] = (lexeme, "ID")          # add entry to symbol table (if not already present)
            tokenval = lexeme
            p-=1
            return ST[lexeme][1]                     # return the token for the identifier lexeme
        
        elif p >= len(m): return DONE                # return DONE Flag for end of iteration
        
        elif isopFirst(c):
            #init
            douRelop = ['<=', '=<', '>=','=>', '==', '!=']
            douOp = ['=+', '+=', '-=', '=-', '*=', '=*', '/=', '=/', '++', '--', '>>', '<<']
            ops = ['+','-','*','/','=', '%', '&', '|', '^', '~', '!', '?']
            rops = ['<','>']
            comm = ['//']
            
            tokenval = c #append to tokenval and checkop
            checkOP =  c
            
            d = getchar(m) # get next token
            checkOP = checkOP + d #append to check for two digit operators
            
            
            if checkOP in douRelop: #check for two character relops
                tokenval=checkOP
                return "RELOP"
            elif checkOP in douOp: #check for two character ops
                tokenval = checkOP
                return "OP"
            elif checkOP in comm:
                while '\n' not in checkOP:
                    c = getchar(m)
                    checkOP += c #append to check for two digit operators
                line_num += 1

                token_val = checkOP
                return "COMMENT"
            else:
                if c in ops: #return single character op
                    p -= 1
                    return "OP"
                elif c in rops: #return single character relop
                    p -= 1
                    return "RELOP"
               
        elif ispun(c):
            tokenval = c
            return "PUN"                             # token is a operator +,-,*, or /
        
        else:
            print("invalid character",c,"found on line",line_num)

    

In [33]:
init()                        # initialize symbol table
                            #test code
m = """ if (x >= 10): {
_count_== "counting"
// this is a comment //
x++ 
x += 1
print("Hello world!")
}
    
    
    
elif: (75 >> =+x)
int float if else for while do break continue goto
+ - * / = += -= *= /= ++ -- % & | ^ >> << ~ ! ?
< > <= >= == !=
( ) { } [ ] ; : ,
A B C D E F G H I J K L M N P Q R S T U V W X Y Z _
0 1 2 3 4 5 6 7 8 9 
"""    # input string to analyze


print("<",lexan(m),",",tokenval,">")        # print < token , value > (using global variable)
while tokenval is not None:                               # while token value is not None
    print("<",lexan(m),",",tokenval,">")    # print < token , value > (using global variable)

< IF , if >
< PUN , ( >
< ID , x >
< RELOP , >= >
< NUM , 10 >
< PUN , ) >
< PUN , : >
< PUN , { >
< ID , _count_ >
< RELOP , == >
invalid character " found on line 2
< ID , counting >
invalid character " found on line 2
< COMMENT , / >
< ID , x >
< OP , ++ >
< ID , x >
< OP , += >
< NUM , 1 >
< ID , print >
< PUN , ( >
invalid character " found on line 6
< ID , Hello >
< ID , world >
< OP , ! >
invalid character " found on line 6
< PUN , ) >
< PUN , } >
< ID , elif >
< PUN , : >
< PUN , ( >
< NUM , 75 >
< OP , >> >
< OP , =+ >
< ID , x >
< PUN , ) >
< INT , int >
< FLOAT , float >
< IF , if >
< ELSE , else >
< FOR , for >
< WHILE , while >
< DO , do >
< BREAK , break >
< CONTINUE , continue >
< GOTO , goto >
< OP , + >
< OP , - >
< OP , * >
< OP , / >
< OP , = >
< OP , += >
< OP , -= >
< OP , *= >
< OP , /= >
< OP , ++ >
< OP , -- >
< OP , % >
< OP , & >
< OP , | >
< OP , ^ >
< OP , >> >
< OP , << >
< OP , ~ >
< OP , ! >
< OP , ? >
< RELOP , < >
< RELOP , > >
< RELOP , <= >
< RELOP , 